# Data Science for Product Managers Final Project

### Data Ingestion and Preparation

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
#data09 = pd.read_excel('DDS9_Data_Extract_with_labels.xlsx', encoding='latin1')
#data10 = pd.read_excel('DDS10_Data_Extract_with_labels.xlsx', encoding='latin1')
#data11 = pd.read_excel('DDS11_Data_Extract_with_labels.xlsx', encoding='latin1')

data09 = pd.read_excel('DDS9_Data_Extract_with_labels.xlsx')
data10 = pd.read_excel('DDS10_Data_Extract_with_labels.xlsx')
data11 = pd.read_excel('DDS11_Data_Extract_with_labels.xlsx')

In [3]:
print(data09.shape)
print(data10.shape)
print(data11.shape)

(2076, 191)
(2205, 197)
(2131, 198)


By printing the shape of the three year's worth of data we can see two important pieces of take-aways:
1. Each of the years has a similar number of observations. This is good as it means we won't be overreliant on one year's data which may have undue influence on the results.
2. We have a different number of features for each year. Having 1/3 or more of observations automatically not having any data for one of the features is a problem.

In order to account for 2, the first step will be to identify which features are not shared between the years.

In [4]:
# shared columns in all 3 years

cols9, cols10, cols11 = list(data09.columns), list(data10.columns), list(data11.columns)
cols910 = [x for x in cols9 if x in cols10]
shared_cols = [x for x in cols910 if x in cols11]
len(shared_cols)

109

In [5]:
# non-shared columns in all 3 years

not_cols910 = [x for x in cols9 if x not in cols10]
not_cols109 = [x for x in cols10 if x not in cols9]
not_cols_comb910 = not_cols910 + not_cols109

not_cols_91011 = [x for x in not_cols_comb910 if x not in cols11]
not_cols_11910 = [x for x in cols11 if x not in not_cols_comb910]

not_cols = not_cols_91011 + not_cols_11910
len(not_cols)

204

In [6]:
# shared columns in 2010 and 2011 data

cols1011 = [x for x in cols10 if x in cols11]
len(cols1011)

182

Here we can see that there are 109 shared columns between all three years and 204 columns that don't share the same name. However, given the similarity in size of years 10 and 11, we assumed that they might have greater similarity. It turns out they did, with 182 columns sharing the exact same name. Therefore, we decided to merge these two years and ignore the data from year 9. 

Before we merged the sets, we had to adjust the names of our target variable in order to make that a shared element. This would then bring a total of 183 shared columns. The target variable of analysis is: "I would be willing to view advertising with my streaming video programming if it significantly reduced the cost of the subscription. (e.g., reduced subscription cost by 25%)"

In [7]:
data10['target_willing'] = data10.iloc[:, -3]
data11['target_willing'] = data11.iloc[:, -3]

The final piece before we merge the datasets is to add a column indicating the year in case that ends up proving relevant in the analysis.

In [8]:
data10['year'] = 10
data11['year'] = 11

In [125]:
merge_cols = [x for x in data10.columns if x in data11.columns]

merge10 = data10[merge_cols]
merge11 = data11[merge_cols]

df = pd.concat([merge10, merge11])

In [126]:
df.head()

,record - Record number,"Q1r1 - To begin, what is your age?",Q4 - What is your gender?,age - you are...,Q2 - In which state do you currently reside?,region - Region,QNEW3 - What is your employment status?,Q5 - Which category best describes your ethnicity?,QNEW1 - Do you have children living in your home (excluding yourself if you are under 18)?,QNEW2 - How old are the children in your home?-0-4 years,...,"Q39rNEW1 - I would rather pay for sports information online in exchange for not being exposed to advertisements. - Using the scale below, please indicate how much you agree or disagree with the following statements. If the question does not apply to you, c","Q39rNEW2 - I would rather pay for games online in exchange for not being exposed to advertisements. - Using the scale below, please indicate how much you agree or disagree with the following statements. If the question does not apply to you, choose ""N/A.""","Q39rNEW3 - I would rather pay for music online in exchange for not being exposed to advertisements. - Using the scale below, please indicate how much you agree or disagree with the following statements. If the question does not apply to you, choose ""N/A.""","Q39rNEW4 - I would rather pay for TV shows online in exchange for not being exposed to advertisements. - Using the scale below, please indicate how much you agree or disagree with the following statements. If the question does not apply to you, choose ""N/A","Q39rNEW5 - I would rather pay for movies online in exchange for not being exposed to advertisements. - Using the scale below, please indicate how much you agree or disagree with the following statements. If the question does not apply to you, choose ""N/A.""","Q39r2 - I would be willing to provide more personal information online if that meant I could receive advertising more targeted to my needs and interests. - Using the scale below, please indicate how much you agree or disagree with the following statements.","Q39r3 - By providing more personal information online, I am worried about becoming a victim of identity theft. - Using the scale below, please indicate how much you agree or disagree with the following statements. If the question does not apply to you, cho",Q89 - Which of the following is your most frequently used mechanism to get news?,target_willing,year
0,7,31,Female,30-46,Illinois,Midwest,Unemployed,White or Caucasian (Non-Hispanic),Yes,No,...,Agree somewhat,Agree strongly,Agree strongly,Agree somewhat,Agree strongly,Agree somewhat,Agree somewhat,Social media sites,Agree somewhat,10
1,4,30,Female,30-46,Arkansas,South,Unemployed,White or Caucasian (Non-Hispanic),Yes,Yes,...,Disagree strongly,Disagree strongly,Disagree strongly,Disagree somewhat,Disagree strongly,Disagree somewhat,Agree somewhat,Social media sites,Agree strongly,10
2,8,61,Male,47-65,Alabama,South,Retired,White or Caucasian (Non-Hispanic),No,NaN,...,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Agree strongly,Television news stations,Agree strongly,10
3,3,68,Female,66 or older,New York,Northeast,Retired,White or Caucasian (Non-Hispanic),No,NaN,...,N/A; I do not have a basis to answer,N/A; I do not have a basis to answer,N/A; I do not have a basis to answer,Disagree strongly,Disagree somewhat,Disagree strongly,Agree strongly,Television news stations,Agree somewhat,10
4,15,50,Female,47-65,Iowa,Midwest,Employed full-time or part-time,White or Caucasian (Non-Hispanic),No,NaN,...,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Agree somewhat,Television news stations,Agree somewhat,10


In [127]:
df.shape

(4336, 184)

The next step is to rename the columns for easier processing.

In [128]:
pd.Series(df.columns).to_csv('cols.csv')

In [129]:
colnames = pd.read_csv('columns.csv')[['Orig', 'New']]

colnames = pd.Series(colnames.New.values, index = colnames.Orig).to_dict()

In [130]:
df = df.rename(columns = colnames)

In [131]:
df.head()

,recNo,age,gender,ageGrp,state,region,status,ethnicity,childAtHome,childAtHome_0-4,...,paySportsforNoAds,payGamesForNoAds,PayMusicForNoAds,PayTVForNoAds,PayMovieForNoAds,infoForTargetAds,infoIDTheft,getNews,target_willing,year
0,7,31,Female,30-46,Illinois,Midwest,Unemployed,White or Caucasian (Non-Hispanic),Yes,No,...,Agree somewhat,Agree strongly,Agree strongly,Agree somewhat,Agree strongly,Agree somewhat,Agree somewhat,Social media sites,Agree somewhat,10
1,4,30,Female,30-46,Arkansas,South,Unemployed,White or Caucasian (Non-Hispanic),Yes,Yes,...,Disagree strongly,Disagree strongly,Disagree strongly,Disagree somewhat,Disagree strongly,Disagree somewhat,Agree somewhat,Social media sites,Agree strongly,10
2,8,61,Male,47-65,Alabama,South,Retired,White or Caucasian (Non-Hispanic),No,NaN,...,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Agree strongly,Television news stations,Agree strongly,10
3,3,68,Female,66 or older,New York,Northeast,Retired,White or Caucasian (Non-Hispanic),No,NaN,...,N/A; I do not have a basis to answer,N/A; I do not have a basis to answer,N/A; I do not have a basis to answer,Disagree strongly,Disagree somewhat,Disagree strongly,Agree strongly,Television news stations,Agree somewhat,10
4,15,50,Female,47-65,Iowa,Midwest,Employed full-time or part-time,White or Caucasian (Non-Hispanic),No,NaN,...,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Disagree strongly,Agree somewhat,Television news stations,Agree somewhat,10


In [132]:
# drop recNo column
df = df.drop(labels = 'recNo', axis = 1)

Next, we will examine the distribution of NAs in the current dataset.

In [133]:
# top 50 columns with missing values 

df.isna().mean().sort_values(ascending=False)[:50]

valueDrone             0.988699
valuePlaceholder       0.986854
valueVR                0.985932
valueSmartwatch        0.974400
valueFitnessband       0.970480
valueE-Reader          0.969788
valueDigTVAntenna      0.966328
valuePortableGaming    0.965406
subsValMagazine        0.936347
valueDVD               0.935194
valueDVR               0.923893
valuePhone             0.905673
subsValNews            0.901983
prefMagazine           0.887915
subsValGaming          0.870387
prefRadio              0.841559
prefNewspaper          0.825876
valueRouter            0.812731
prefLiveShow           0.809041
valueTablet            0.807887
subsValMobileVoice     0.775138
subsValLandline        0.775138
prefMovie              0.730166
prefBook               0.725323
prefVideoGames         0.721172
subsValMobileData      0.697878
valueDesktop           0.646679
valueTV                0.645756
prefMusic              0.624769
childAtHome_14-18      0.614391
childAtHome_19-25      0.614391
childAtH

Details about dropped columns and rationale

Post inspection of the columns, we will be removing columns that do not add value to our analysis. Following is the set of columns to be removed and rationale behind each:
1. State column as we will be using 'Region' for our segmentation purposes 
2. Columns containing age of the children - 'How old are the children in your home?'
These columns have been removed because these contain lot of null values , and for our segmentation analysis- the information that a family has children or not is enough and age group bifurcation is not relevant.
3. Columns containing Ranking of how important an owned media or home equipment is for a family- 
Of the products you indicated you own, which do you value the most?
These columns have very high proportion of null values, and the rank assignment is very subjective to individual user and not a good candidate for generalisation -  A '2' for one user doesn't mean the same as a '2' for another user
4. Columns covering 'Don't Know' answer option - These columns do not add value to the analysis
What types of apps do you use frequently (everyday/weekly) on your smartphone?-Don't Know
Which of the following subscriptions does your household purchase?-Don't Know
5. Columns containing Ranking of how valued an owned subscription is for a family-
Of the services you indicated your household purchases, which do you value the most?
These columns have very high proportion of null values, and the rank assignment is very subjective to individual user and not a good candidate for generalisation -  A '2' for one user doesn't mean the same as a '2' for another user 
6. Columns identifying mediums in which customers are willing to avail paid services in lieu of no advertisements-
I would rather pay for X in exchange for not being exposed to advertisements
These set of columns are other similar outcome variables to our target. Given that these are more output than input related, these would not add value to our analysis

In [134]:
#Dropping certain columns based on rationale described in above cell
cols_keep = pd.read_csv('columns.csv')[['New', 'Drop']]
cols_keep= cols_keep[cols_keep['Drop']!=1]
col_list= list(cols_keep['New'])
cols = [x for x in df.columns if x in col_list]
df=df[cols]
df.shape

(4336, 142)

Next, we will convert answers to numbers for easier processing later in our models. For Yes / No columns we replaced the values with 1 / 0, and for the other values an additional column labelled 'coded' 
- (Household income specific) Less than \\$29,999 = 0, \\$30,000 to \\$ 49,999 = 1, \\$ 50,000 to \\$ 99,999 = 2, \\$ 100,000 to \\$ 299,999 = 3, More than \\$ 300,000 = 4, Do not know = Null value
- Yes = 1, No = 0
- Never = 0, Rarely (one to three times a year) = 1, Occasionally (monthly) = 2, Frequently (every day/weekly) = 3
- Almost never = 0, Rarely (10%-50% of the time) = 1, Frequently (between 50% and 75% of the time) = 2, Almost always (greater than 75% of the time) = 3, Always (close to 100% of the time) = 4
- N/A; I do not have a basis to answer = Null value, Disagree strongly = 0, Disagree somewhat = 1, Agree somewhat = 2, Agree strongly = 3

A copy of the original data will be created so that the original responses are saved for reference.

In [232]:
df2 = df.copy()

In [233]:
df2.replace(('Yes','No'),(1,0), inplace=True)

In [234]:
# Question 19

cols = ['freqMovRentDVD','freqMovBuyDVD','freqMovBuyDownload','freqMovRentDownload',
    'freqMovStream','freqMovPayPerView','freqTVRentDVD','freqTVBuyDVD', 
    'freqTVBuyDownload','freqTVStream','freqTVPayPerView']

df2[cols] = df2[cols].replace(['Never', 'Rarely (one to three times a year)', 'Occasionally (monthly)','Frequently (every day/weekly)'], [0, 1, 2, 3])

In [235]:
# Question 73

cols = ['TVWhileReadWork','TVWhileReadPleasure','TVWhileWeb','TVWhileMicroblog',
    'TVWhileReadEmail','TVWhileWriteEmail','TVWhileText','TVWhileSocial',
   'TVWhileCall','TVWhileBrowseProd','TVWhilePurchaseProd','TVWhileVideoGames']

df2[cols] = df2[cols].replace(['Almost never', 'Rarely (10%-50% of the time)', 'Frequently (between 50% and 75% of the time)','Almost always (greater than 75% of the time)','Always (close to 100% of the time)'], [0, 1, 2, 3, 4])

In [236]:
# Question 39 and target column

cols = ['infoForTargetAds','infoIDTheft', 'target_willing']

df2[cols] = df2[cols].replace(['N/A; I do not have a basis to answer', 'Disagree strongly', 'Disagree somewhat','Agree somewhat','Agree strongly'], [np.nan, 0, 1, 2, 3])

In [237]:
# Household Income

df2[['HHIncome']] = df2[['HHIncome']].replace(['Less than $29,999', '$30,000 to $49,999', '$50,000 to $99,999','$100,000 to $299,999','More than $300,000','Do not know'], [0, 1, 2, 3, 4, np.nan])

In [238]:
df2.head()

,age,gender,region,status,ethnicity,childAtHome,HHIncome,ownsTV,ownsDVR,ownsStreamingBox,...,TVWhileCall,TVWhileBrowseProd,TVWhilePurchaseProd,TVWhileVideoGames,infoForTargetAds,infoIDTheft,getNews,target_willing,year,prefMovieTV
0,31,Female,Midwest,Unemployed,White or Caucasian (Non-Hispanic),1,2.0,0,0,0,...,3,4,3,4,2.0,2.0,Social media sites,2.0,10,0
1,30,Female,South,Unemployed,White or Caucasian (Non-Hispanic),1,1.0,1,1,0,...,1,1,1,0,1.0,2.0,Social media sites,3.0,10,0
2,61,Male,South,Retired,White or Caucasian (Non-Hispanic),0,3.0,1,1,0,...,1,1,1,1,0.0,3.0,Television news stations,3.0,10,0
3,68,Female,Northeast,Retired,White or Caucasian (Non-Hispanic),0,2.0,1,0,1,...,1,1,0,1,0.0,3.0,Television news stations,2.0,10,0
4,50,Female,Midwest,Employed full-time or part-time,White or Caucasian (Non-Hispanic),0,2.0,1,1,0,...,1,1,1,2,0.0,2.0,Television news stations,2.0,10,0


Next, we will convert our categorical variables into a form that will better allow us to run our models on them.  This will be done with one hot encoding, and specifically the fields: gender, region, ethnicity, and employment status.  These fields will have new columns appended to the dataframe that will align with each variable option for the field (e.g. Gender becomes two columns - gender_Female and gender_Male).

In [239]:
df2.select_dtypes(exclude = np.number)

,gender,region,status,ethnicity,willPayDoubleInternet,getNews
0,Female,Midwest,Unemployed,White or Caucasian (Non-Hispanic),I am willing to pay $20 per month on top of wh...,Social media sites
1,Female,South,Unemployed,White or Caucasian (Non-Hispanic),I am willing to pay $10 per month on top of wh...,Social media sites
2,Male,South,Retired,White or Caucasian (Non-Hispanic),I am willing to pay $10 per month on top of wh...,Television news stations
3,Female,Northeast,Retired,White or Caucasian (Non-Hispanic),I am not willing to pay more for faster downlo...,Television news stations
4,Female,Midwest,Employed full-time or part-time,White or Caucasian (Non-Hispanic),I am not willing to pay more for faster downlo...,Television news stations
...,...,...,...,...,...,...
2126,Male,West,Unemployed,White or Caucasian (Non-Hispanic),I am willing to pay $20 per month on top of wh...,Social media sites
2127,Male,Northeast,Retired,White or Caucasian (Non-Hispanic),I am willing to pay $5 per month on top of wha...,Television news stations
2128,Male,South,Unemployed,White or Caucasian (Non-Hispanic),I am willing to pay $10 per month on top of wh...,Television news stations
2129,Female,Midwest,Retired,White or Caucasian (Non-Hispanic),I am not willing to pay more for faster downlo...,Print newspapers


To simplify column names, we will first recode several categorical values:

Q3 - What is your employment status?
- Employed full-time or part-time $\rightarrow$ Employed

Q89 - Which of the following is your most frequently used mechanism to get news?

- Social media sites $\rightarrow$ Social Media
- Television news stations $\rightarrow$ TV News
- Online news sites not associated with a newspaper $\rightarrow$ Online Non-Newspaper
- I do not follow the news $\rightarrow$ No News
- Online news sites not associated with a traditional news organization $\rightarrow$ Online Non-News

Q29 - You said that you subscribe to home Internet access, how much more would you be willing to pay to receive double your download speed?

- I am willing to pay \\$5 per month on top of what I already pay $\rightarrow$ plus5 
- I am willing to pay \\$10 per month on top of what I already pay $\rightarrow$ plus10
- I am willing to pay \\$20 per month on top of what I already pay $\rightarrow$ plus20
- I am willing to pay \\$30 or more per month on top of what I already pay $\rightarrow$ plus30 
- I am not willing to pay more for faster download speeds as my current speed is sufficient for my needs $\rightarrow$ sufficient
- I prefer faster speed but I am unwilling to pay more than I already do $\rightarrow$ unwilling

Q5 - Which category best describes your ethnicity?

- White or Caucasian (Non-Hispanic) $\rightarrow$ White (note that a separate category for Hispanic is provided)
- South Asian (India, Pakistan, Sri Lanka $\rightarrow$ South Asian
- East Asian or Southeast Asian' $\rightarrow$ E_SE Asian
- Pacific Islander (including Native Hawaiian, Native American, or Native Alaskan) $\rightarrow$ Pac Islander
- Something else not listed here $\rightarrow$ Other

In [240]:
df2['status'].replace('Employed full-time or part-time', 'Employed', inplace=True)

In [241]:
df2['getNews'].replace(['Social media sites', 'Television news stations', 'Online news sites not associated with a newspaper', 'I do not follow the news', 'Online news sites not associated with a traditional news organization'], 
                       ['social media', 'TV news', 'online non-newspaper', 'no news', 'online non-news'],  inplace = True)

In [242]:
df2['willPayDoubleInternet'].replace(['I am willing to pay $5 per month on top of what I already pay', 'I am willing to pay $10 per month on top of what I already pay', 'I am willing to pay $20 per month on top of what I already pay', 'I am willing to pay $30 or more per month on top of what I already pay', 'I am not willing to pay more for faster download speeds as my current speed is sufficient for my needs', 'I prefer faster speed but I am unwilling to pay more than I already do'],
                                    ['plus5', 'plus10', 'plus20', 'plus30', 'sufficient', 'unwilling'], inplace = True)

In [243]:
df2['ethnicity'].replace(['White or Caucasian (Non-Hispanic)', 'South Asian (India, Pakistan, Sri Lanka)', 'East Asian or Southeast Asian', 'Pacific Islander (including Native Hawaiian, Native American, or Native Alaskan)', 'Something else not listed here'], 
                        ['white', 'South Asian', 'E_SE Asian', 'Pac Islander', 'Other'], inplace = True)

In [244]:
# create dummies for all non-numeric columns

cols = df2.select_dtypes(exclude = np.number).columns
dummies = pd.get_dummies(df2[cols])

In [245]:
# add dummy columns to dataframe

df2 = pd.concat([df2, dummies], axis=1)

In [246]:
# drop original columns 

df2.drop(columns = cols, inplace = True)

In [247]:
df2.shape

(4336, 175)

In [248]:
df2.head()

,age,childAtHome,HHIncome,ownsTV,ownsDVR,ownsStreamingBox,ownsStreamingDrive,ownsDigTVAntenna,ownsDVD,ownsGaming,...,getNews_News aggregators,getNews_Online version of newspapers,getNews_Print newspapers,getNews_Radio,getNews_TV news,getNews_Variety/talk shows,getNews_no news,getNews_online non-news,getNews_online non-newspaper,getNews_social media
0,31,1,2.0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,30,1,1.0,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
2,61,0,3.0,1,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,68,0,2.0,1,0,1,1,0,1,1,...,0,0,0,0,1,0,0,0,0,0
4,50,0,2.0,1,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


Next, additional feature engineering will be performed to reduce the dimensions. Question 37 asks: For the following types of entertainment activities, please rank your top three, with "1" being the most preferred.

As we are interested in media that could be streamed, we will create a column for "prefMovieTV" as 1 for anyone who has ranked going to the movies or watching television as one of their top 3 preferred entertainment activities, and 0 otherwise.

In [249]:
df2['prefMovieTV'] = 0
df2.loc[(df2['prefMovie']>0)| (df2['prefTV']>0), 'prefMovieTV'] = 1

In [250]:
drop = [c for c in df2.columns if "pref" in c and c != "prefMovieTV"]

In [251]:
drop

['prefLiveShow',
 'prefMovie',
 'prefTV',
 'prefMusic',
 'prefBook',
 'prefMagazine',
 'prefNewspaper',
 'prefRadio',
 'prefVideoGames',
 'prefSocial']

In [252]:
df2.drop(columns = drop, inplace = True)

In [253]:
df2.shape

(4336, 165)

Similarly, question 22 asks: What types of apps do you use frequently (everyday/weekly) on your smartphone? 

As we are interested in video streaming, we will create a new variable called "useVideoApps" that is set to 1 if the respondent indicated that they use movie or Video apps frequently, and 0 otherwise.

In [254]:
df2['useVideoApps'] = 0
df2.loc[(df2['appVideo']>0)| (df2['appMovie']>0), 'prefMovieTV'] = 1

In [255]:
drop = [c for c in df2.columns if "app" in c and c != "useVideoApps"]
print(drop)
df2.drop(columns = drop, inplace = True)

['appPhotoVideo', 'appBanking', 'appFitness', 'appFoodDrink', 'appRetail', 'appNavigation', 'appSocial', 'appReading', 'appMusic', 'appVideo', 'appReviews', 'appNewsAggregator', 'appNewspaper', 'appMagazines', 'appDiagnosticUtilities', 'appQRReader', 'appTravel', 'appTransportation', 'appLocation', 'appLanguage', 'appMovie', 'appBusiness', 'appGames', 'appProductivity', 'appSports', 'appWeather', 'appBrowser', 'appVOIP', 'appDating', 'appMessaging', 'appMobilePayment', 'appEducation', 'appTickets', 'appReservations', 'appHobby', 'appNA ']


Next, additional null values will be dealt with. Since our target variable "target_willing" will be used for prediction, we will drop any rows with NA values in this column.

In [256]:
df2.dropna(subset=['target_willing'], inplace=True)

In [257]:
# around 750 rows were dropped
df2.shape

(3584, 130)

In [258]:
df2.isna().sum().sort_values(ascending = False)

HHIncome            220
infoForTargetAds    121
infoIDTheft          66
useVideoApps          0
movieTablet           0
                   ... 
gender_Male           0
region_Midwest        0
region_Northeast      0
region_South          0
age                   0
Length: 130, dtype: int64

There are around 200 NA values in the HHIncome column, 121 in teh infoForTargetAds column, 66 in the infoIDTheft column. Since the total number of rows is quite small, we will remove these rows from the dataset.

In [259]:
df2.dropna(inplace = True)

In [260]:
df2.shape

(3222, 130)

We'll now look at explaining and visualizing the updated and cleaned data that we have.

In [146]:
# Displaying the summary statistics for each column value to have a better sense of the data
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,4336.0,42.678967,19.982386,14.0,25.0,38.0,64.0,99.0
gender_Female,4336.0,0.501384,0.500056,0.0,0.0,1.0,1.0,1.0
gender_Male,4336.0,0.498616,0.500056,0.0,0.0,0.0,1.0,1.0
childAtHome,4336.0,0.385609,0.486795,0.0,0.0,0.0,1.0,1.0
HHIncomeCoded,4046.0,1.516560,1.063194,0.0,1.0,2.0,2.0,4.0
...,...,...,...,...,...,...,...,...
status_Employed full-time or part-time,4336.0,0.421356,0.493833,0.0,0.0,0.0,1.0,1.0
status_Retired,4336.0,0.233625,0.423185,0.0,0.0,0.0,0.0,1.0
status_Self-employed,4336.0,0.071264,0.257295,0.0,0.0,0.0,0.0,1.0
status_Student,4336.0,0.143681,0.350806,0.0,0.0,0.0,0.0,1.0
